# SBERT 학습과 활용

저희는 `klue/roberta-base` 모델을 **KLUE** 내 **STS** 데이터셋을 활용하여 모델을 훈련하는 방식을 택했습니다.

SBERT를 학습하는 방법 중 문장 쌍으로 회귀 문제를 푸는 STS 문제를 푸는 것을 목표로 모델을 구축하였습니다. STS란 두 개의 문장으로부터 의미적 유사성을 구하는 문제를 뜻합니다.

학습을 통해 얻어질 `sentence-klue-roberta-base` 모델은 입력된 문장의 임베딩을 계산해 유사도를 예측하는 데 사용할 수 있습니다.

학습 과정 이후에는 큐레이션과 유사한 문장 순서로 기사를 정렬하는 방향으로 모델을 활용하였습니다.


소스 코드는 [`sentence-transformers`](https://github.com/UKPLab/sentence-transformers) 원 라이브러리를 참고하였습니다.

먼저, 노트북을 실행하는 데 필요한 라이브러리를 설치합니다. 모델 훈련을 위해서는 `sentence-transformers`가, 학습 데이터셋 로드를 위해서는 `datasets` 라이브러리의 설치가 필요합니다.


In [ ]:
!pip install sentence-transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model_name = "klue/roberta-base"

본 노트북에서는 `klue-roberta-base` 모델을 활용하지만, https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

In [ ]:
train_batch_size = 8
num_epochs = 4

모델 정보 외에도 학습에 필요한 하이퍼 파라미터를 정의합니다.

In [ ]:
from sentence_transformers import SentenceTransformer, losses, models, util

앞서 정의한 사전학습 언어 모델을 로드합니다.

`sentence-transformers`는 HuggingFace의 `transformers`와 호환이 잘 이루어지고 있기 때문에, [모델 허브](https://huggingface.co/models)에 올라와있는 대부분의 언어 모델을 임베딩을 추출할 *Embedder* 로 사용할 수 있습니다.

In [ ]:
embedding_model =  models.Transformer(model_name)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

In [ ]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens= True,
    # 문장의 (속) 단어들의 평균으로 문장을 표현. 모든 단어가 선택
    # BERT의 모든 출력 벡터들을 평균낸다 (MEAN pooling)
    pooling_mode_cls_token = False,
    # [CLS] 토큰의 출력 벡터를 문장 벡터로 간주하는 것이다. -> 따라서 [CLS] 토큰의 출력 벡터는 문장의 문맥을 모두 참고한, 문맥을 반영한 임베딩이 된다
    # cls라는 특수토큰을 만들어서 문장을 표현. CLS 토큰은 classification token으로 전체 시퀀스를 분류하는 의미를 지니게 된다.
    pooling_mode_max_tokens= False,
    # max_token 문장의 단어들 중 max token으로 (최대값) 문장을 표현. 하나만 선택
    # 각 단어 벡터의 dimension 중 가장 큰 값을 선택(MAX pooling)하는 것
)

*Embedder* 에서 추출된 토큰 단위 임베딩들을 가지고 문장 임베딩을 어떻게 계산할 것인지를 결정하는 *Pooler* 를 정의합니다.

여러 Pooling 기법이 있겠지만, 저희는 토큰화(Tokenizer)를 진행할 때 **Mean Pooling**을 사용하는 방법을 택했습니다.

**Mean Pooling**이란 모델이 반환한 모든 토큰 임베딩을 더해준 후, 더해진 토큰 개수만큼 나누어 문장을 대표하는 임베딩으로 사용하는 기법을 의미합니다.

*Embedder* 와 *Pooler* 를 정의했으므로, 이 두 모듈로 구성된 하나의 모델을 정의합니다.

`modules`에 입력으로 들어가는 모듈이 순차적으로 임베딩 과정에 사용됩니다.

In [ ]:
model = SentenceTransformer(modules=[embedding_model, pooler])

# 학습에 사용할 데이터 로드하기


뉴스 기사를 크롤링한 후 직접 유사도 묶음을 짝지어 저희 모델에 학습시킬 유사도 데이터를 구축하였습니다.

그리고 데이터 전처리 과정에서 콘텐츠상 피해야 하거나 필요 및 의미가 없는 요소들을 불용어로 지정하는 작업을 추가로 진행하여 저희 데이터에 맞는 불용어 리스트를 만들었습니다.

이렇게 만들어 낸 데이터셋을 로드해 모델 학습에 이용하였습니다.

In [ ]:
import pandas as pd

my_train = pd.read_excel("/content/주가지수_train_filtered.xlsx")
my_val = pd.read_excel("/content/주가지수_val_filtered.xlsx")

 데이터셋은 학습을 위해 훈련용(Training) 데이터셋과 검증용(Validation) 데이터 셋을 분리해서 로드시켰습니다.

데이터는 아래와 같이 두 개의 문장과 두 문장의 유사도를 라벨로 지니고 있습니다.

In [ ]:
my_train.head()

,page1,page2,label
0,"코스피', '코스닥', '지수', '나란히', '넘게', '하락', '마감', '했...","호실', '주식', '베팅', '개미', '투자', '개월', '만에', '작년',...",5
1,"코스피', '코스닥', '지수', '나란히', '넘게', '하락', '마감', '했...","바이', '코리아', '정연주', '제작', '일러스트', '외국인', '투자자',...",5
2,"코스피', '코스닥', '지수', '나란히', '넘게', '하락', '마감', '했...","지난달', '국내', '외국인', '투자자', '상장', '주식', '채권', '가...",5
3,"코스피', '코스닥', '지수', '나란히', '넘게', '하락', '마감', '했...","금감원', '외인', '개월', '만에', '주식', '매수', '전환', '지난'...",5
4,"코스피', '코스닥', '지수', '나란히', '넘게', '하락', '마감', '했...","이후', '가장', '많아', '노르웨이', '매수', '외국인', '투자자', '...",5


이제 앞서 불러온 데이터셋을 `sentence-transformers` 훈련 양식에 맞게 변환해주는 작업을 거쳐야 합니다.

두 데이터 모두 0점에서 5점 사이의 값으로 유사도가 기록되었기 때문에, 0.0 ~ 1.0 스케일로 정규화를 시켜주는 작업을 거치게 됩니다.

In [ ]:
from sentence_transformers.readers import InputExample

train_samples = []
for i in range(len(my_train)):  # 5*6
  train_samples.append(
      InputExample(  # 열 부분 0 , 1, 2 -> sen1, sen2, label(유사도)
          texts = [my_train.iloc[i, 0], # sentence1:0
                   my_train.iloc[i, 1], # sentence2:1
                   ],
          label = float(my_train.iloc[i, 2]/5.0),  # /5.0로 나누는 것 = # 0.0 ~ 1.0 스케일로 유사도 정규화
      )
  )



valid_samples = []
for i in range(len(my_val)):  # 2*6
  valid_samples.append(
      InputExample(  # 열 부분 0 , 1, 2 -> sen1, sen2, label(유사도)
          texts = [my_val.iloc[i, 0], # sentence1:0
                   my_val.iloc[i, 1], # sentence2:1
                   ],
          label = float(my_val.iloc[i, 2]/5.0),
      )
  )

앞선 로직을 통해 각 데이터 예제는 다음과 같이 `InputExample` 객체로 변환되게 됩니다.

In [ ]:
train_samples[0].texts, train_samples[0].label

(["코스피', '코스닥', '지수', '나란히', '넘게', '하락', '마감', '했다', '지난', '뉴욕', '주가지수', '일제', '급락', '하며', '국내', '증시', '대한', '투자', '심리', '약해졌다', '초반', '국내외', '기관', '투자자', '강한', '동반', '도세', '지속', '되며', '지수', '하방', '압력', '받았', '다미', '연방', '준비', '제도', '준가', '강도', '높은', '기준금리', '인상', '양적', '긴축', '예고', '하며', '달러화', '가치', '상승세', '지속', '하고', '있는', '가운데', '동아시아', '주요', '증시', '흐름', '엇갈렸다', '우리나라', '중국', '증시', '하락', '반면', '일본', '증시', '외국인', '관광객', '입국', '재개', '적극', '추진', '하겠다고', '밝히며', '상승', '마감', '했다', '오후', '서울', '중구', '을지로', '하나은행', '본점', '딜링룸', '모습', '연합뉴스', '오후', '서울', '중구', '을지로', '하나은행', '본점', '딜링룸', '모습', '연합뉴스', '코스피지수', '전날', '보다', '포인트', '내린', '마감', '했다', '오전', '리기', '했으나', '개인', '투자자', '매수세가', '낙폭', '확대', '방어', '했다', '한국', '거래소', '따르면', '하루', '개인', '유가', '증권', '시장', '어치', '사들였다', '지난', '이후', '거래', '만에', '가장', '수액', '개인', '가장', '많이', '사들인', '종목', '억원', '수액', '기록', '했다', '주가', '대폭', '하락', '카카오', '네이버', '주식', '억원', '억원', '매수', '했다', '카카오', '네이버', '전날', '보다', '내렸다', '반면', '외국인', '강한', '도세', '이어', '갔다', 

In [ ]:
valid_samples[0].texts, valid_samples[0].label

(['일 코스피코스닥지수가 나란히  넘게 하락 마감했다 지난 밤 미 뉴욕 대 주가지수가 일제히 급락하며 국내 증시에 대한 투자 심리도 약해졌다 장 초반부터 국내외 기관 투자자들의 강한 동반 매도세가 지속되며 지수가 하방 압력을 받았다미 연방준비제도연준가 강도 높은 기준금리 인상과 양적 긴축을 예고하며 달러화 가치가 상승세를 지속하고 있는 가운데 동아시아 주요국들의 증시 흐름은 다소 엇갈렸다 우리나라와 중국 증시는 큰폭으로 하락한 반면 일본 증시는 다음 달 중 외국인 관광객의 입국 재개를 적극 추진하겠다고 밝히며 상승 마감했다일 오후 서울 중구 을지로 하나은행 본점 딜링룸의 모습 연합뉴스일 오후 서울 중구 을지로 하나은행 본점 딜링룸의 모습 연합뉴스이날 코스피지수는 전날보다 포인트 내린 로 마감했다 오전 중 까지 내리기도 했으나 개인 투자자의 매수세가 낙폭 확대를 방어했다한국거래소에 따르면 이날 하루 동안 개인은 유가증권시장에서 총 억원어치를 사들였다 지난 달 일 이후 거래일 만에 가장 큰 순매수액이다 개인이 가장 많이 사들인 종목은 삼성전자원   로 억원의 순매수액을 기록했다 주가가 대폭 하락한 카카오원   와 네이버원    주식도 각각 억원 억원 순매수했다 이날 카카오와 네이버는 각각 전날보다   내렸다반면 외국인은 여전히 강한 매도세를 이어갔다 유가증권시장 현물 시장에서 총 억원을 코스피 선물 시장에서 억원을 순매도했다 외국인은 개인 투자자들이 많이 산 종목을 대량 매도했다 삼성전자를 억원 카카오를 억원 네이버를 억원어치 팔았다국내 기관은 현물 시장에서 억원을 순매도했으며 선물 시장에서는 외국인 매물 억원어치를 사들였다 국내 기관의 매도세 역시 삼성전자와 카카오 네이버에 몰렸다코스피지수뿐 아니라 기술 성장주 중심의 코스닥지수도 대폭 하락했다 이날 코스닥지수는 전날보다 포인트 내린 로 마감했다 외국인과 국내 기관이 각각 억원 억원을 순매도했으며 개인은 억원어치를 사들였다코스닥시장에서는 특히 게임 관련주들이 큰폭으로 내렸다 펄어비스원   가  위메이드원   가

이제 학습에 사용될 `DataLoader`와 **Loss**를 설정해주도록 합니다.

`CosineSimilarityLoss`는 입력된 두 문장의 임베딩 간 코사인 유사도와 골드 라벨 간 차이를 통해 계산되게 됩니다.

In [ ]:
import torch
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    train_samples, shuffle=True, batch_size = train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

모델 검증에 활용할 **Evaluator** 를 정의해줍니다.

앞서 얻어진 검증 데이터를 활용하여, 모델의 문장 임베딩 간 코사인 유사도가 얼마나 골드 라벨에 가까운지 계산하는 역할을 수행합니다.

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    valid_samples,
    name="sts-valid"
)

모델 학습에 사용될 **Warm up Steps**를 설정합니다.

다양한 방법으로 스텝 수를 결정할 수 있겠지만, 예제 노트북에서는 원 예제 코드를 따라 훈련 배치 수의 10% 만큼으로 값을 설정합니다.

In [ ]:
import math
warmup_steps = math.ceil(len(train_dataloader)* num_epochs * 0.1)

이제 앞서 얻어진 객체, 값들을 가지고 모델의 훈련을 진행합니다.

`sentence-transformers`에서는 다음과 같이 `fit` 함수를 통해 간단히 모델의 훈련과 검증이 가능합니다.

훈련 과정을 통해 매 에폭 마다 얻어지는 체크포인트에 대해 *Evaluator* 가 학습된 모델의 코사인 유사도와 골드 라벨 간 피어슨, 스피어만 상관 계수를 계산해 기록을 남기게 됩니다.

In [ ]:
model.fit(
    train_objectives = [(train_dataloader, train_loss)],
    evaluator = evaluator,
    epochs = num_epochs,
    evaluation_steps = 1000,
    warmup_steps = warmup_steps,
    output_path = model_save_path
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

# 학습 시킨 모델을 저장하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_save_path = "/content/drive/MyDrive/Colab Notebooks/juga"

In [ ]:
torch.save (model.state_dict(), model_save_path+'/jugajisu.pt')

추론을 위해 모델을 저장할 때는 그 모델의 학습된 매개변수만 저장(학습시킨 가중치만 저장)하면 됩니다. torch.save() 를 사용하여 모델의 state_dict 를 저장하는 것이 나중에 모델을 사용할 때 가장 유연하게 사용할 수 있는, 모델 저장 시 권장하는 방법입니다.

# Sentence Transformers 활용

입력된 문장 간 유사도를 쉽고 빠르게 구할 수 있도록 설계된 `sentence-transformers`를 이용한다면 임베딩을 활용해 다양한 어플리케이션을 고안할 수 있습니다.

먼저 여러 문장 후보군이 주어졌을 때, 입력된 문장과 가장 유사한 문장을 계산하는 예제를 살펴보도록 합시다.

이를 위해 검색의 대상이 되는 문장 후보군을 다음과 같이 정의할 필요가 있습니다. 이후, 정의된 문장 후보군을 미리 임베딩합니다.

In [ ]:
# 인출을 위한 후보군
docs = [
    "코스피가 전장보다 0.89p(0.04%) 오른 2480.24에 마쳤다.지수는 전장보다 15.05p(0.61%) 오른 2494.40에 개장해 강세를 이어가다가 점차 상승 폭을 줄였다. 이후 오후 한때는 약보합권으로 돌아서기도 했다.",
    "10일 한국거래소에 따르면 지난 한 달(4월 7일~5월 9일) 동안 외국인 투자자는 코스피 시장에서 3조6176억원어치를 순매수하며 매수 우위를 보였다. 반면 이 기간 개인투자자는 코스피에서 2조9221억원어치를 팔아치웠다. 기관 투자자도 6659억원어치를 순매도했다.",
    " 금융소비자 개개인의 상황과 조건에 따른 가장 합리적인 상품을 제공하고 편의성을 높여 최적화된 금융 서비스를 제공한다.",
    "피겨 주니어 그랑프리에 데뷔하던 2019년, 14살 중학생 이해인은 쇼팽의 야상곡 20번 올림 다단조를 쇼트프로그램 음악으로 선택해 많은 사람을 놀라게 했다. 보통 사람들에게도 친숙한 쇼팽의 야상곡 2번은 경쾌하고 발랄함을 표현하려는 어린 선수들이 피겨 음악으로 많이 사용해왔지만, 야상곡 20번은 웬만한 선수들이 소화하기 어려운 음악으로 알려져 있기 때문이다.",
    "더불어민주당을 탈당한 김남국 의원의 코인 거래 의혹을 정조준한 검찰이 수사에 본격 착수했다. 지난 15일 서울남부지검 형사6부(부장검사 이준동)는 업비트, 빗썸, 카카오의 블록체인 관련 계열사 세 곳을 압수수색해 김 의원의 ‘위믹스’ 등 가상자산 거래 내역 등을 확보한 것으로 전해졌다.",
    "르세라핌은 지난 1일 발매된 첫 정규 앨범 '언포기븐'에 대해 '우리는 매번 성장하고 있고 이러한 성장과 긍정적인 변화가 모든 앨범에 담기도록 노력한다. 우리가 전하고 싶은 메시지가 곧 르세라핌이기 때문'이라며 '타인의 시선이나 평가에 개의치 않는 모습이 누군가에게는 빌런처럼 보일 수도 있다. 하지만 우리의 음악이 선을 넘고 경계를 넘어 확장되는 것처럼, 계속해서 한계를 넘어서는 팀이 되고 싶다'고 설명했다."
]

docs_embeddings = model.encode(docs)

이제 입력 문장을 임베딩 할 차례입니다.

In [ ]:
# 쿼리가 질문
query = """지수는 전장보다 4.10p(0.16%) 내린 2505.96에 개장한 뒤 낙폭을 키워 장중 2488.42까지 내렸다. 다만 곧 반등해 약보합권에서 마쳤다."""
query_embedding = model.encode(query)

아래는 입력된 문장의 임베딩과 미리 임베딩 된 후보군 문장 임베딩 간 유사도를 계산해 유사도가 높은 순서대로 `top_k` 개 문장을 뽑아주는 예제 코드입니다.

`top_k`는 전체 문장 후보군의 개수를 넘지 않아야 하므로, `min()` 함수를 통해 예외 처리를 해줍니다.

In [ ]:
# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, docs_embeddings)[0]
# 코사인 유사도 순으로 `top_k` 개 문장 추출
tok_k = 4
top_results = torch.topk(cos_scores, k=tok_k)
for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
  print(f"{i}번째 결과: {docs[idx]}, 유사도는 {score} \n")

0번째 결과: 코스피가 전장보다 0.89p(0.04%) 오른 2480.24에 마쳤다.지수는 전장보다 15.05p(0.61%) 오른 2494.40에 개장해 강세를 이어가다가 점차 상승 폭을 줄였다. 이후 오후 한때는 약보합권으로 돌아서기도 했다., 유사도는 0.9648637175559998 

1번째 결과: 10일 한국거래소에 따르면 지난 한 달(4월 7일~5월 9일) 동안 외국인 투자자는 코스피 시장에서 3조6176억원어치를 순매수하며 매수 우위를 보였다. 반면 이 기간 개인투자자는 코스피에서 2조9221억원어치를 팔아치웠다. 기관 투자자도 6659억원어치를 순매도했다., 유사도는 0.716131865978241 

2번째 결과: 더불어민주당을 탈당한 김남국 의원의 코인 거래 의혹을 정조준한 검찰이 수사에 본격 착수했다. 지난 15일 서울남부지검 형사6부(부장검사 이준동)는 업비트, 빗썸, 카카오의 블록체인 관련 계열사 세 곳을 압수수색해 김 의원의 ‘위믹스’ 등 가상자산 거래 내역 등을 확보한 것으로 전해졌다., 유사도는 0.524139940738678 

3번째 결과: 피겨 주니어 그랑프리에 데뷔하던 2019년, 14살 중학생 이해인은 쇼팽의 야상곡 20번 올림 다단조를 쇼트프로그램 음악으로 선택해 많은 사람을 놀라게 했다. 보통 사람들에게도 친숙한 쇼팽의 야상곡 2번은 경쾌하고 발랄함을 표현하려는 어린 선수들이 피겨 음악으로 많이 사용해왔지만, 야상곡 20번은 웬만한 선수들이 소화하기 어려운 음악으로 알려져 있기 때문이다., 유사도는 0.45091456174850464 



# 저장한 모델 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, losses, models, util
model_name = "klue/roberta-base"
embedding_model = models.Transformer(model_name)
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False

)

model = SentenceTransformer (modules=[embedding_model,pooler])

device = torch.device('cpu')
model.load_state_dict(torch.load("realasset.pt", map_location=device))